---
title: Man vs Machine
---

In [ ]:
import logging
import os

if not os.getenv(
    "NBGRADER_EXECUTION"
):
    %reload_ext jupyter_ai

::::{figure} https://upload.wikimedia.org/wikipedia/commons/thumb/2/28/Terminator_%28franchise_logo%29.png/512px-Terminator_%28franchise_logo%29.png
:alt: Terminator (franchise logo)
::::

In this notebook, you will compete with your classmates and your machine by

1. handcrafting a decision tree using Weka `UserClassifier`, and
2. using `python-weka-wrapper` to build the J48 (C4.5) decision tree as a comparison.

Let's find out who is the most intelligent!

## Interactive Decision Tree Construction

Follow the instruction above [\[Witten11\] Ex 17.2.12][Witten11] to 

1. install the package `UserClassifier`,
2. hand-build a decision tree using [`segment-challenge.arff`][segment-challenge] as the training set, and 
3. test the performance using [`segment-test.arff`][segment-test] as the test set.

[Witten11]: https://ebookcentral.proquest.com/lib/cityuhk/reader.action?docID=634862&ppg=604
[segment-challenge]: https://raw.githubusercontent.com/Waikato/weka-3.8/master/wekadocs/data/segment-challenge.arff
[segment-test]: https://raw.githubusercontent.com/Waikato/weka-3.8/master/wekadocs/data/segment-test.arff

You may also install version 1.0.3 of the userClassifier (available on [SourceForge](https://sourceforge.net/projects/weka/files/weka-packages/)) directly by running the following code cell, or manually download the zip file and install it using Weka's package manager.

In [ ]:
import logging
import os

if not os.getenv("NBGRADER_EXECUTION"):
    import weka.core.jvm as jvm
    import weka.core.packages as packages

    jvm.start(packages=True, logging_level=logging.ERROR)
    pkg, version = "userClassifier", "1.0.3"
    if not packages.is_installed(pkg):
        print(f"Installing {pkg}...")
        packages.install_package("userClassifier", version="1.0.3")
        print("Done.")
    else:
        print(f"Skipping {pkg}, already installed.")

::::{exercise}
:label: ex:1
 Why should you NOT use the training set `segment-challenge.arff` to test your decision tree?
::::

YOUR ANSWER HERE

::::{exercise}
:label: ex:2
 Why should you NOT use the class attribute to split the decision tree?
::::

YOUR ANSWER HERE

Get ready to dive into the thrilling world of decision trees! It’s time to showcase your data science prowess and outshine your classmates. Here’s what you need to do:

::::{exercise}
:label: ex:3

Include the model and result summary sections from the result buffer of your best hand-built decision tree. Your answer should look like:

```
=== Classifier model (full training set) ===

Split on ...

Time taken to build model: ...

=== Confusion Matrix ===

...

=== Summary ===

Correctly Classified Instances ...

```

Try your best to beat your classmates and the machines:
- Build at least two decision trees and pick the best one. 
- Share your result (and knowledge) on the [discussion page on Interactive Decision Tree Construction](https://canvas.cityu.edu.hk/courses/67502/discussion_topics/597573).
- See if your classmates have posted better decision trees and give them a like if they have.

If you find a good strategy (algorithm) to build good trees all the time, please also include it with your answer below.


::::

YOUR ANSWER HERE

::::{exercise}
:label: ex:4
For the best decision tree you hand-built, is the empirical accuracy on the test set an unbiased estimate of your performance? Explain the direction of the bias, if any.
::::

YOUR ANSWER HERE

In [ ]:
%%ai
I am in a competition to hand build the best decision tree using the 
UserClassifier package of Weka. Can you describe in one paragraph how to use
the scatter plots to find pairs of attributes to split? I cannot do detailed
calculations. How to avoid overfitting?

## Python Weka Wrapper

To see if your hand-built classifier can beat the machine, use J48 (C4.5) to build a decision tree. Instead of using the Weka Explorer Interface, you will run Weka directly from the notebook using [`python-weka-wrapper3`][pww].

[pww]: http://fracpete.github.io/python-weka-wrapper3/

Because Weka is written in Java, we need to start the java virtual machine first.

In [ ]:
import weka.core.jvm as jvm
import logging

jvm.start(logging_level=logging.ERROR)

### Loading dataset

To load the dataset, create an `ArffLoader` as follows:

In [ ]:
from weka.core.converters import Loader

loader = Loader(classname="weka.core.converters.ArffLoader")

The loader has the method `load_url` to load data from the web, such as the [Weka GitLab repository][weka]:

[weka]: https://git.cms.waikato.ac.nz/weka/weka/-/tree/stable-3-8/wekadocs/data

In [ ]:
weka_data_path = (
    "https://git.cms.waikato.ac.nz/weka/weka/-/raw/stable-3-8/wekadocs/data/"
)
trainset = loader.load_url(
    weka_data_path + "segment-challenge.arff"
)  # use load_file to load from file instead

For classification, we have to specify the class attribute. For instance, the method `class_is_last` mutates `trainset` to have the last attribute as the class attribute:

In [ ]:
trainset.class_is_last()

::::{exercise}
:label: ex:5

1. Assign `testset` to the data loaded from `weka_data_path + 'segment-test.arff'`. 
2. Specify the last attribute of `testset` as the target.

::::

In [ ]:
from weka.core.dataset import Instances

# YOUR CODE HERE
raise NotImplementedError
print(Instances.summary(testset))

In [ ]:
# tests
assert testset.relationname == "segment"
assert testset.num_instances == 810
assert testset.num_attributes == 20

In [ ]:
# hidden tests

### Training using J48

To train a decision tree using J48, we create the classifier and then apply the method `build_classifier` on the training set.

In [ ]:
from weka.classifiers import Classifier

J48 = Classifier(classname="weka.classifiers.trees.J48")
J48.build_classifier(trainset)
J48

To visualize the tree by generating an SVG file:

In [ ]:
if not os.getenv(
    "NBGRADER_EXECUTION"
):
    !mamba install --yes pygraphviz
    import pygraphviz as pgv
    from IPython.display import SVG
    
    # Create a PyGraphviz AGraph object from the DOT data
    pgv.AGraph(string=J48.graph).draw('J48tree.svg', prog='dot')
    
    # Display the SVG file
    SVG(filename="J48tree.svg")

::::{tip} How to edit the decision tree?
:class: dropdown

`J48.graph` is a piece of code written in a domain-specific language called [DOT graph][dot]. You can save the dot file instead of the rendered image, so that you can edit it further. To do so:

1. Save the string to a text file such as [`J48tree.gv`](J48tree.gv)

2. Edit/preview it in vscode using the [extension](https://marketplace.visualstudio.com/items?itemName=tintinweb.graphviz-interactive-preview&ssr=false#version-history). To install the extension:  

    1. Run the command in a terminal:

      ```bash
      install-vscode-extension tintinweb.graphviz-interactive-preview@0.3.5
      ```

    2. Reload the vscode window with the command `> Developer: Reload Window`.

There are also online editors available such as:

:::::{card}
:header: [open in new tab](https://dreampuf.github.io/GraphvizOnline/#digraph%20J48Tree%20%7B%0AN0%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN0-%3EN1%20%5Blabel%3D%22%3C%3D%20155%22%5D%0AN1%20%5Blabel%3D%22value-mean%22%20%5D%0AN1-%3EN2%20%5Blabel%3D%22%3C%3D%2091.4444%22%5D%0AN2%20%5Blabel%3D%22rawred-mean%22%20%5D%0AN2-%3EN3%20%5Blabel%3D%22%3C%3D%2024.6667%22%5D%0AN3%20%5Blabel%3D%22hue-mean%22%20%5D%0AN3-%3EN4%20%5Blabel%3D%22%3C%3D%20-1.89048%22%5D%0AN4%20%5Blabel%3D%22hue-mean%22%20%5D%0AN4-%3EN5%20%5Blabel%3D%22%3C%3D%20-2.22266%22%5D%0AN5%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN5-%3EN6%20%5Blabel%3D%22%3C%3D%20146%22%5D%0AN6%20%5Blabel%3D%22foliage%20(102.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN5-%3EN7%20%5Blabel%3D%22%3E%20146%22%5D%0AN7%20%5Blabel%3D%22cement%20(3.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN4-%3EN8%20%5Blabel%3D%22%3E%20-2.22266%22%5D%0AN8%20%5Blabel%3D%22rawred-mean%22%20%5D%0AN8-%3EN9%20%5Blabel%3D%22%3C%3D%202.55556%22%5D%0AN9%20%5Blabel%3D%22hue-mean%22%20%5D%0AN9-%3EN10%20%5Blabel%3D%22%3C%3D%20-2.09121%22%5D%0AN10%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN10-%3EN11%20%5Blabel%3D%22%3C%3D%20129%22%5D%0AN11%20%5Blabel%3D%22foliage%20(50.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN10-%3EN12%20%5Blabel%3D%22%3E%20129%22%5D%0AN12%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN12-%3EN13%20%5Blabel%3D%22%3C%3D%20128%22%5D%0AN13%20%5Blabel%3D%22rawred-mean%22%20%5D%0AN13-%3EN14%20%5Blabel%3D%22%3C%3D%200.666667%22%5D%0AN14%20%5Blabel%3D%22foliage%20(30.0%2F4.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN13-%3EN15%20%5Blabel%3D%22%3E%200.666667%22%5D%0AN15%20%5Blabel%3D%22window%20(5.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN12-%3EN16%20%5Blabel%3D%22%3E%20128%22%5D%0AN16%20%5Blabel%3D%22vedge-mean%22%20%5D%0AN16-%3EN17%20%5Blabel%3D%22%3C%3D%200.333333%22%5D%0AN17%20%5Blabel%3D%22window%20(11.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN16-%3EN18%20%5Blabel%3D%22%3E%200.333333%22%5D%0AN18%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN18-%3EN19%20%5Blabel%3D%22%3C%3D%20216%22%5D%0AN19%20%5Blabel%3D%22window%20(3.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN18-%3EN20%20%5Blabel%3D%22%3E%20216%22%5D%0AN20%20%5Blabel%3D%22foliage%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN9-%3EN21%20%5Blabel%3D%22%3E%20-2.09121%22%5D%0AN21%20%5Blabel%3D%22window%20(38.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN8-%3EN22%20%5Blabel%3D%22%3E%202.55556%22%5D%0AN22%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN22-%3EN23%20%5Blabel%3D%22%3C%3D%20121%22%5D%0AN23%20%5Blabel%3D%22exgreen-mean%22%20%5D%0AN23-%3EN24%20%5Blabel%3D%22%3C%3D%20-15.4444%22%5D%0AN24%20%5Blabel%3D%22brickface%20(2.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN23-%3EN25%20%5Blabel%3D%22%3E%20-15.4444%22%5D%0AN25%20%5Blabel%3D%22vedge-mean%22%20%5D%0AN25-%3EN26%20%5Blabel%3D%22%3C%3D%202.94444%22%5D%0AN26%20%5Blabel%3D%22window%20(75.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN25-%3EN27%20%5Blabel%3D%22%3E%202.94444%22%5D%0AN27%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN27-%3EN28%20%5Blabel%3D%22%3C%3D%20134%22%5D%0AN28%20%5Blabel%3D%22cement%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN27-%3EN29%20%5Blabel%3D%22%3E%20134%22%5D%0AN29%20%5Blabel%3D%22window%20(8.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN22-%3EN30%20%5Blabel%3D%22%3E%20121%22%5D%0AN30%20%5Blabel%3D%22rawred-mean%22%20%5D%0AN30-%3EN31%20%5Blabel%3D%22%3C%3D%207.88889%22%5D%0AN31%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN31-%3EN32%20%5Blabel%3D%22%3C%3D%2043%22%5D%0AN32%20%5Blabel%3D%22brickface%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN31-%3EN33%20%5Blabel%3D%22%3E%2043%22%5D%0AN33%20%5Blabel%3D%22window%20(13.0%2F2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN30-%3EN34%20%5Blabel%3D%22%3E%207.88889%22%5D%0AN34%20%5Blabel%3D%22saturation-mean%22%20%5D%0AN34-%3EN35%20%5Blabel%3D%22%3C%3D%200.492526%22%5D%0AN35%20%5Blabel%3D%22cement%20(15.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN34-%3EN36%20%5Blabel%3D%22%3E%200.492526%22%5D%0AN36%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN36-%3EN37%20%5Blabel%3D%22%3C%3D%2082%22%5D%0AN37%20%5Blabel%3D%22foliage%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN36-%3EN38%20%5Blabel%3D%22%3E%2082%22%5D%0AN38%20%5Blabel%3D%22cement%20(4.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN3-%3EN39%20%5Blabel%3D%22%3E%20-1.89048%22%5D%0AN39%20%5Blabel%3D%22exgreen-mean%22%20%5D%0AN39-%3EN40%20%5Blabel%3D%22%3C%3D%20-4.77778%22%5D%0AN40%20%5Blabel%3D%22vedge-mean%22%20%5D%0AN40-%3EN41%20%5Blabel%3D%22%3C%3D%202.77778%22%5D%0AN41%20%5Blabel%3D%22brickface%20(198.0%2F2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN40-%3EN42%20%5Blabel%3D%22%3E%202.77778%22%5D%0AN42%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN42-%3EN43%20%5Blabel%3D%22%3C%3D%20115%22%5D%0AN43%20%5Blabel%3D%22brickface%20(4.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN42-%3EN44%20%5Blabel%3D%22%3E%20115%22%5D%0AN44%20%5Blabel%3D%22foliage%20(3.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN39-%3EN45%20%5Blabel%3D%22%3E%20-4.77778%22%5D%0AN45%20%5Blabel%3D%22hedge-mean%22%20%5D%0AN45-%3EN46%20%5Blabel%3D%22%3C%3D%200.833336%22%5D%0AN46%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN46-%3EN47%20%5Blabel%3D%22%3C%3D%20115%22%5D%0AN47%20%5Blabel%3D%22foliage%20(4.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN46-%3EN48%20%5Blabel%3D%22%3E%20115%22%5D%0AN48%20%5Blabel%3D%22window%20(42.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN45-%3EN49%20%5Blabel%3D%22%3E%200.833336%22%5D%0AN49%20%5Blabel%3D%22grass%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN2-%3EN50%20%5Blabel%3D%22%3E%2024.6667%22%5D%0AN50%20%5Blabel%3D%22hue-mean%22%20%5D%0AN50-%3EN51%20%5Blabel%3D%22%3C%3D%20-2.17742%22%5D%0AN51%20%5Blabel%3D%22vedge-mean%22%20%5D%0AN51-%3EN52%20%5Blabel%3D%22%3C%3D%205%22%5D%0AN52%20%5Blabel%3D%22window%20(4.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN51-%3EN53%20%5Blabel%3D%22%3E%205%22%5D%0AN53%20%5Blabel%3D%22foliage%20(18.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN50-%3EN54%20%5Blabel%3D%22%3E%20-2.17742%22%5D%0AN54%20%5Blabel%3D%22rawgreen-mean%22%20%5D%0AN54-%3EN55%20%5Blabel%3D%22%3C%3D%2024.4444%22%5D%0AN55%20%5Blabel%3D%22brickface%20(3.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN54-%3EN56%20%5Blabel%3D%22%3E%2024.4444%22%5D%0AN56%20%5Blabel%3D%22cement%20(180.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN1-%3EN57%20%5Blabel%3D%22%3E%2091.4444%22%5D%0AN57%20%5Blabel%3D%22sky%20(220.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN0-%3EN58%20%5Blabel%3D%22%3E%20155%22%5D%0AN58%20%5Blabel%3D%22exgreen-mean%22%20%5D%0AN58-%3EN59%20%5Blabel%3D%22%3C%3D%20-2%22%5D%0AN59%20%5Blabel%3D%22saturation-mean%22%20%5D%0AN59-%3EN60%20%5Blabel%3D%22%3C%3D%200.385555%22%5D%0AN60%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN60-%3EN61%20%5Blabel%3D%22%3C%3D%20159%22%5D%0AN61%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN61-%3EN62%20%5Blabel%3D%22%3C%3D%20208%22%5D%0AN62%20%5Blabel%3D%22cement%20(3.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN61-%3EN63%20%5Blabel%3D%22%3E%20208%22%5D%0AN63%20%5Blabel%3D%22path%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN60-%3EN64%20%5Blabel%3D%22%3E%20159%22%5D%0AN64%20%5Blabel%3D%22path%20(234.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN59-%3EN65%20%5Blabel%3D%22%3E%200.385555%22%5D%0AN65%20%5Blabel%3D%22cement%20(11.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN58-%3EN66%20%5Blabel%3D%22%3E%20-2%22%5D%0AN66%20%5Blabel%3D%22grass%20(205.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0A%7D)
::::{iframe} https://dreampuf.github.io/GraphvizOnline/#digraph%20J48Tree%20%7B%0AN0%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN0-%3EN1%20%5Blabel%3D%22%3C%3D%20155%22%5D%0AN1%20%5Blabel%3D%22value-mean%22%20%5D%0AN1-%3EN2%20%5Blabel%3D%22%3C%3D%2091.4444%22%5D%0AN2%20%5Blabel%3D%22rawred-mean%22%20%5D%0AN2-%3EN3%20%5Blabel%3D%22%3C%3D%2024.6667%22%5D%0AN3%20%5Blabel%3D%22hue-mean%22%20%5D%0AN3-%3EN4%20%5Blabel%3D%22%3C%3D%20-1.89048%22%5D%0AN4%20%5Blabel%3D%22hue-mean%22%20%5D%0AN4-%3EN5%20%5Blabel%3D%22%3C%3D%20-2.22266%22%5D%0AN5%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN5-%3EN6%20%5Blabel%3D%22%3C%3D%20146%22%5D%0AN6%20%5Blabel%3D%22foliage%20(102.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN5-%3EN7%20%5Blabel%3D%22%3E%20146%22%5D%0AN7%20%5Blabel%3D%22cement%20(3.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN4-%3EN8%20%5Blabel%3D%22%3E%20-2.22266%22%5D%0AN8%20%5Blabel%3D%22rawred-mean%22%20%5D%0AN8-%3EN9%20%5Blabel%3D%22%3C%3D%202.55556%22%5D%0AN9%20%5Blabel%3D%22hue-mean%22%20%5D%0AN9-%3EN10%20%5Blabel%3D%22%3C%3D%20-2.09121%22%5D%0AN10%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN10-%3EN11%20%5Blabel%3D%22%3C%3D%20129%22%5D%0AN11%20%5Blabel%3D%22foliage%20(50.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN10-%3EN12%20%5Blabel%3D%22%3E%20129%22%5D%0AN12%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN12-%3EN13%20%5Blabel%3D%22%3C%3D%20128%22%5D%0AN13%20%5Blabel%3D%22rawred-mean%22%20%5D%0AN13-%3EN14%20%5Blabel%3D%22%3C%3D%200.666667%22%5D%0AN14%20%5Blabel%3D%22foliage%20(30.0%2F4.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN13-%3EN15%20%5Blabel%3D%22%3E%200.666667%22%5D%0AN15%20%5Blabel%3D%22window%20(5.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN12-%3EN16%20%5Blabel%3D%22%3E%20128%22%5D%0AN16%20%5Blabel%3D%22vedge-mean%22%20%5D%0AN16-%3EN17%20%5Blabel%3D%22%3C%3D%200.333333%22%5D%0AN17%20%5Blabel%3D%22window%20(11.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN16-%3EN18%20%5Blabel%3D%22%3E%200.333333%22%5D%0AN18%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN18-%3EN19%20%5Blabel%3D%22%3C%3D%20216%22%5D%0AN19%20%5Blabel%3D%22window%20(3.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN18-%3EN20%20%5Blabel%3D%22%3E%20216%22%5D%0AN20%20%5Blabel%3D%22foliage%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN9-%3EN21%20%5Blabel%3D%22%3E%20-2.09121%22%5D%0AN21%20%5Blabel%3D%22window%20(38.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN8-%3EN22%20%5Blabel%3D%22%3E%202.55556%22%5D%0AN22%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN22-%3EN23%20%5Blabel%3D%22%3C%3D%20121%22%5D%0AN23%20%5Blabel%3D%22exgreen-mean%22%20%5D%0AN23-%3EN24%20%5Blabel%3D%22%3C%3D%20-15.4444%22%5D%0AN24%20%5Blabel%3D%22brickface%20(2.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN23-%3EN25%20%5Blabel%3D%22%3E%20-15.4444%22%5D%0AN25%20%5Blabel%3D%22vedge-mean%22%20%5D%0AN25-%3EN26%20%5Blabel%3D%22%3C%3D%202.94444%22%5D%0AN26%20%5Blabel%3D%22window%20(75.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN25-%3EN27%20%5Blabel%3D%22%3E%202.94444%22%5D%0AN27%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN27-%3EN28%20%5Blabel%3D%22%3C%3D%20134%22%5D%0AN28%20%5Blabel%3D%22cement%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN27-%3EN29%20%5Blabel%3D%22%3E%20134%22%5D%0AN29%20%5Blabel%3D%22window%20(8.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN22-%3EN30%20%5Blabel%3D%22%3E%20121%22%5D%0AN30%20%5Blabel%3D%22rawred-mean%22%20%5D%0AN30-%3EN31%20%5Blabel%3D%22%3C%3D%207.88889%22%5D%0AN31%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN31-%3EN32%20%5Blabel%3D%22%3C%3D%2043%22%5D%0AN32%20%5Blabel%3D%22brickface%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN31-%3EN33%20%5Blabel%3D%22%3E%2043%22%5D%0AN33%20%5Blabel%3D%22window%20(13.0%2F2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN30-%3EN34%20%5Blabel%3D%22%3E%207.88889%22%5D%0AN34%20%5Blabel%3D%22saturation-mean%22%20%5D%0AN34-%3EN35%20%5Blabel%3D%22%3C%3D%200.492526%22%5D%0AN35%20%5Blabel%3D%22cement%20(15.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN34-%3EN36%20%5Blabel%3D%22%3E%200.492526%22%5D%0AN36%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN36-%3EN37%20%5Blabel%3D%22%3C%3D%2082%22%5D%0AN37%20%5Blabel%3D%22foliage%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN36-%3EN38%20%5Blabel%3D%22%3E%2082%22%5D%0AN38%20%5Blabel%3D%22cement%20(4.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN3-%3EN39%20%5Blabel%3D%22%3E%20-1.89048%22%5D%0AN39%20%5Blabel%3D%22exgreen-mean%22%20%5D%0AN39-%3EN40%20%5Blabel%3D%22%3C%3D%20-4.77778%22%5D%0AN40%20%5Blabel%3D%22vedge-mean%22%20%5D%0AN40-%3EN41%20%5Blabel%3D%22%3C%3D%202.77778%22%5D%0AN41%20%5Blabel%3D%22brickface%20(198.0%2F2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN40-%3EN42%20%5Blabel%3D%22%3E%202.77778%22%5D%0AN42%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN42-%3EN43%20%5Blabel%3D%22%3C%3D%20115%22%5D%0AN43%20%5Blabel%3D%22brickface%20(4.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN42-%3EN44%20%5Blabel%3D%22%3E%20115%22%5D%0AN44%20%5Blabel%3D%22foliage%20(3.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN39-%3EN45%20%5Blabel%3D%22%3E%20-4.77778%22%5D%0AN45%20%5Blabel%3D%22hedge-mean%22%20%5D%0AN45-%3EN46%20%5Blabel%3D%22%3C%3D%200.833336%22%5D%0AN46%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN46-%3EN47%20%5Blabel%3D%22%3C%3D%20115%22%5D%0AN47%20%5Blabel%3D%22foliage%20(4.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN46-%3EN48%20%5Blabel%3D%22%3E%20115%22%5D%0AN48%20%5Blabel%3D%22window%20(42.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN45-%3EN49%20%5Blabel%3D%22%3E%200.833336%22%5D%0AN49%20%5Blabel%3D%22grass%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN2-%3EN50%20%5Blabel%3D%22%3E%2024.6667%22%5D%0AN50%20%5Blabel%3D%22hue-mean%22%20%5D%0AN50-%3EN51%20%5Blabel%3D%22%3C%3D%20-2.17742%22%5D%0AN51%20%5Blabel%3D%22vedge-mean%22%20%5D%0AN51-%3EN52%20%5Blabel%3D%22%3C%3D%205%22%5D%0AN52%20%5Blabel%3D%22window%20(4.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN51-%3EN53%20%5Blabel%3D%22%3E%205%22%5D%0AN53%20%5Blabel%3D%22foliage%20(18.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN50-%3EN54%20%5Blabel%3D%22%3E%20-2.17742%22%5D%0AN54%20%5Blabel%3D%22rawgreen-mean%22%20%5D%0AN54-%3EN55%20%5Blabel%3D%22%3C%3D%2024.4444%22%5D%0AN55%20%5Blabel%3D%22brickface%20(3.0%2F1.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN54-%3EN56%20%5Blabel%3D%22%3E%2024.4444%22%5D%0AN56%20%5Blabel%3D%22cement%20(180.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN1-%3EN57%20%5Blabel%3D%22%3E%2091.4444%22%5D%0AN57%20%5Blabel%3D%22sky%20(220.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN0-%3EN58%20%5Blabel%3D%22%3E%20155%22%5D%0AN58%20%5Blabel%3D%22exgreen-mean%22%20%5D%0AN58-%3EN59%20%5Blabel%3D%22%3C%3D%20-2%22%5D%0AN59%20%5Blabel%3D%22saturation-mean%22%20%5D%0AN59-%3EN60%20%5Blabel%3D%22%3C%3D%200.385555%22%5D%0AN60%20%5Blabel%3D%22region-centroid-row%22%20%5D%0AN60-%3EN61%20%5Blabel%3D%22%3C%3D%20159%22%5D%0AN61%20%5Blabel%3D%22region-centroid-col%22%20%5D%0AN61-%3EN62%20%5Blabel%3D%22%3C%3D%20208%22%5D%0AN62%20%5Blabel%3D%22cement%20(3.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN61-%3EN63%20%5Blabel%3D%22%3E%20208%22%5D%0AN63%20%5Blabel%3D%22path%20(2.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN60-%3EN64%20%5Blabel%3D%22%3E%20159%22%5D%0AN64%20%5Blabel%3D%22path%20(234.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN59-%3EN65%20%5Blabel%3D%22%3E%200.385555%22%5D%0AN65%20%5Blabel%3D%22cement%20(11.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0AN58-%3EN66%20%5Blabel%3D%22%3E%20-2%22%5D%0AN66%20%5Blabel%3D%22grass%20(205.0)%22%20shape%3Dbox%20style%3Dfilled%20%5D%0A%7D
::::

[dot]: https://en.wikipedia.org/wiki/DOT_(graph_description_language)

### Evaluation

To evaluate the decision tree on the training set:

In [ ]:
from weka.classifiers import Evaluation

J48train = Evaluation(trainset)
J48train.test_model(J48, trainset)
train_accuracy = J48train.percent_correct
print(f"Training accuracy: {train_accuracy:.4g}%")

::::{exercise}
:label: ex:6
 Assign `test_accuracy` to the accuracy of the classifier on the test set.
::::

In [ ]:
# YOUR CODE HERE
raise NotImplementedError
print(f"Test accuracy: {test_accuracy:.4g}%")

In [ ]:
# hidden tests

::::{exercise}
:label: ex:7
 By comparing the training accuracy and test accuracy, can we tell whether a classifier overfitted the data?
::::

YOUR ANSWER HERE

To stop the Java virtual machine, run the following line. To restart `jvm`, you must restart the kernel.

In [ ]:
jvm.stop()